# Title

In [ ]:
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'  # always print last expr.
%config InlineBackend.figure_format = 'svg'
%load_ext autoreload
%autoreload 2
%matplotlib inline

import pandas

pandas.options.display.max_rows = 6

In [ ]:
from tsdm.datasets import BaseDataset

In [ ]:
class USHCN(BaseDataset):
    url = "https://cdiac.ess-dive.lbl.gov/ftp/ushcn_daily/"


# USHCN.download()

# State Codes

In [ ]:
# Best viewed with elastic tabstops!
from io import StringIO

STATE_CODES = r"""
ID	Abbr.	State
01	AL	Alabama
02	AZ	Arizona
03	AR	Arkansas
04	CA	California
05	CO	Colorado
06	CT	Connecticut
07	DE	Delaware
08	FL	Florida
09	GA	Georgia
10	ID	Idaho
11	IL	Idaho
12	IN	Indiana
13	IA	Iowa
14	KS	Kansas
15	KY	Kentucky
16	LA	Louisiana
17	ME	Maine
18	MD	Maryland
19	MA	Massachusetts
20	MI	Michigan
21	MN	Minnesota
22	MS	Mississippi
23	MO	Missouri
24	MT	Montana
25	NE	Nebraska
26	NV	Nevada
27	NH	NewHampshire
28	NJ	NewJersey
29	NM	NewMexico
30	NY	NewYork
31	NC	NorthCarolina
32	ND	NorthDakota
33	OH	Ohio
34	OK	Oklahoma
35	OR	Oregon
36	PA	Pennsylvania
37	RI	RhodeIsland
38	SC	SouthCarolina
39	SD	SouthDakota
40	TN	Tennessee
41	TX	Texas
42	UT	Utah
43	VT	Vermont
44	VA	Virginia
45	WA	Washington
46	WV	WestVirginia
47	WI	Wisconsin
48	WY	Wyoming
"""


state_dtypes = {
    "ID": pandas.CategoricalDtype(ordered=True),
    "Abbr.": pandas.CategoricalDtype(ordered=True),
    "State": pandas.StringDtype(),
}
states = pandas.read_csv(StringIO(STATE_CODES), sep="\t", dtype=state_dtypes)

In [ ]:
states.to_parquet("a.pq")
states2 = pandas.read_parquet("a.pq")
pandas.testing.assert_frame_equal(states, states2)

# Stations Meta-Data

In [ ]:
station_colspecs = {
    "COOP_ID": (1, 6),
    "LATITUDE": (8, 15),
    "LONGITUDE": (17, 25),
    "ELEVATION": (27, 32),
    "STATE": (34, 35),
    "NAME": (37, 66),
    "COMPONENT_1": (68, 73),
    "COMPONENT_2": (75, 80),
    "COMPONENT_3": (82, 87),
    "UTC_OFFSET": (89, 90),
}

# fix colspec to 0-index, half open interval
station_colspecs = {key: (a - 1, b) for key, (a, b) in station_colspecs.items()}

station_dtypes = {
    "COOP_ID": pandas.CategoricalDtype(ordered=True),
    "LATITUDE": pandas.Float32Dtype(),
    "LONGITUDE": pandas.Float32Dtype(),
    "ELEVATION": pandas.Float32Dtype(),
    "STATE": states.ID.dtype,
    "NAME": pandas.StringDtype(),
    "COMPONENT_1": pandas.CategoricalDtype(ordered=True),
    "COMPONENT_2": pandas.CategoricalDtype(ordered=True),
    "COMPONENT_3": pandas.CategoricalDtype(ordered=True),
    "UTC_OFFSET": "timedelta64[h]",
}

station_na_values = {
    "ELEVATION": -999.9,
    "COMPONENT_1": "------",
    "COMPONENT_2": "------",
    "COMPONENT_3": "------",
}

In [ ]:
stations_filename = "ushcn-stations.txt"
stations_filepath = USHCN.rawdata_dir.joinpath(stations_filename)
stations = pandas.read_fwf(
    stations_filepath,
    na_values=station_na_values,
    colspecs=list(station_colspecs.values()),
    header=0,
    names=station_colspecs,
    dtype=station_dtypes,
)
COOP_IDS = pandas.CategoricalDtype(stations.COOP_ID, ordered=True)
stations.astype(
    {
        "COOP_ID": COOP_IDS,
        "COMPONENT_1": COOP_IDS,
        "COMPONENT_2": COOP_IDS,
        "COMPONENT_3": COOP_IDS,
    }
)

# Station Data

In [ ]:
MFLAGS = pandas.CategoricalDtype(("B", "D", "H", "K", "L", "O", "P", "T", "W"))
QFLAGS = pandas.CategoricalDtype(
    ("D", "G", "I", "K", "L", "M", "N", "O", "R", "S", "T", "W", "X", "Z")
)
SFLAGS = pandas.CategoricalDtype(
    (
        "0",
        "6",
        "7",
        "A",
        "B",
        "F",
        "G",
        "H",
        "K",
        "M",
        "N",
        "R",
        "S",
        "T",
        "U",
        "W",
        "X",
        "Z",
    )
)
ELEMENTS = pandas.CategoricalDtype(("PRCP", "SNOW", "SNWD", "TMAX", "TMIN"))


dtypes = {
    "COOP_ID": COOP_IDS,
    "YEAR": pandas.UInt16Dtype(),
    "MONTH": pandas.UInt8Dtype(),
    "ELEMENT": ELEMENTS,
    "VALUE": pandas.Int16Dtype(),
    "MFLAG": MFLAGS,
    "QFLAG": QFLAGS,
    "SFLAG": SFLAGS,
}

# column start, stop, dtype
colspecs = {
    "COOP_ID": (1, 6),
    "YEAR": (7, 10),
    "MONTH": (11, 12),
    "ELEMENT": (13, 16),
}

for k, i in enumerate(range(17, 258, 8)):
    colspecs |= {
        ("VALUE", k + 1): (i, i + 4),
        ("MFLAG", k + 1): (i + 5, i + 5),
        ("QFLAG", k + 1): (i + 6, i + 6),
        ("SFLAG", k + 1): (i + 7, i + 7),
    }

    # dtype |= {
    #     f"VALUE-{k+1}" : integer,
    #     f"MFLAG-{k+1}" : mflag_types,
    #     f"QFLAG-{k+1}" : qflag_types,
    #     f"SFLAG-{k+1}" : sflag_types,
    # }


# These should coincide with the description in data_format.txt
widths = [b - a + 1 for a, b in colspecs.values()]
dtype = {
    key: (dtypes[key[0]] if isinstance(key, tuple) else dtypes[key]) for key in colspecs
}

cspec = [(a - 1, b - 1) for a, b in colspecs.values()]
na_values = {("VALUE", k): -9999 for k in range(1, 32)}
# ds = pandas.read_fwf("state32.txt", names=colspecs, widths=widths, header=None, dtype=dtype, na_values=-9999)

In [ ]:
from zipfile import ZipFile
import gzip

fname = "state01_AL.txt"
fpath = USHCN.rawdata_dir.joinpath("us.txt")

In [ ]:
from modin import pandas as pd

In [ ]:
%%time
ds = pd.read_fwf(
    fpath, names=colspecs, widths=widths, header=None, dtype=dtype, na_values=-9999
)

In [ ]:
times = ds[["YEAR", "MONTH"]]

In [ ]:
pd.to_datetime(times, errors="coerce")

In [ ]:
daskframe.from_pandas(ds._to_pandas(), chunksize=12)

In [ ]:
%%time
with gzip.open(fpath) as file:
    ds = pandas.read_fwf(
        file, names=colspecs, widths=widths, header=None, dtype=dtype, na_values=-9999
    )

ds

In [ ]:
ds[("QFLAG", 1)].fill_na

# preprocessing the data

In [ ]:
id_cols = ["COOP_ID", "YEAR", "MONTH", "ELEMENT"]
data_cols = ["VALUE", "MFLAG", "QFLAG", "SFLAG"]
data_cols = [col for col in ds.columns if col not in id_cols]
columns = pandas.MultiIndex.from_tuples(ds[data_cols], names=["VAR", "DAY"])
data = pandas.DataFrame(ds[data_cols], columns=columns)
data.index.name = "INDEX"
data

In [ ]:
%%time
# Pure magic https://stackoverflow.com/a/27044843/9318372
data = data.stack(level="DAY", dropna=False).reset_index(level="DAY")

In [ ]:
%%time
data = ds[id_cols].join(data, how="inner").reset_index()
data = data.astype(dtypes | {"DAY": integer})
data = data[
    ["COOP_ID", "YEAR", "MONTH", "DAY", "ELEMENT", "MFLAG", "QFLAG", "SFLAG", "VALUE"]
]

In [ ]:
%%time
mask = pandas.isna(data[["MFLAG", "QFLAG", "SFLAG", "VALUE"]]).sum(axis=1) < 4
data = data[mask]
data = data.sort_values(by=["YEAR", "MONTH", "DAY", "COOP_ID", "ELEMENT"]).reset_index(
    drop=True
)
data

# ALternative: Use Modin for speedup

In [ ]:
import os

os.environ["MODIN_ENGINE"] = "ray"
from modin import pandas as pd

In [ ]:
import sys

In [ ]:
{"ray", "modin"} <= sys.modules.keys()

In [ ]:
import os
import ray

In [ ]:
ray.init(num_cpus=os.cpu_count() - 2)

os.environ["MODIN_ENGINE"] = "ray"  # Modin will use Ray
# os.environ["MODIN_ENGINE"] = "dask"  # Modin will use Dask

In [ ]:
# problem: currently only works uncompressed.

from modin import pandas as pd

fname = "us.txt"
fpath2 = USHCN.rawdata_path.joinpath(fname)

In [ ]:
%%time
ds = pd.read_fwf(
    fpath2, names=colspecs, widths=widths, header=None, na_values=-9999, dtype=dtype
)

In [ ]:
id_cols = ["COOP_ID", "YEAR", "MONTH", "ELEMENT"]
data_cols = ["VALUE", "MFLAG", "QFLAG", "SFLAG"]
data_cols = [col for col in ds.columns if col not in id_cols]
columns = pd.MultiIndex.from_tuples(ds[data_cols], names=["VAR", "DAY"])
data = pd.DataFrame(ds[data_cols])
data.columns = columns
data

In [ ]:
%%time
# Pure magic https://stackoverflow.com/a/27044843/9318372
data = data.stack(level="DAY", dropna=True).reset_index(level="DAY")

In [ ]:
%%time
_dtypes = {k: v for k, v in dtypes.items() if k in data.columns} | {
    "DAY": pandas.UInt8Dtype()
}
data = data.astype(_dtypes)

In [ ]:
%%time
data = ds[id_cols].join(data, how="inner")

In [ ]:
data.info()

In [ ]:
data[["COOP_ID"]].info()

In [ ]:
data = data.reset_index(drop=True)

In [ ]:
data.to_parquet(USHCN.dataset_file)

In [ ]:
%%time
data = data[
    ["COOP_ID", "YEAR", "MONTH", "DAY", "ELEMENT", "MFLAG", "QFLAG", "SFLAG", "VALUE"]
]
data = data.sort_values(by=["YEAR", "MONTH", "DAY", "COOP_ID", "ELEMENT"]).reset_index(
    drop=True
)

In [ ]:
%%time
df2 = pd.read_feather(USHCN.dataset_path.joinpath("USHCN.feather"))

In [ ]:
%%time
df2 = pd.read_parquet(USHCN.dataset_path.joinpath("USHCN.parquet"))

In [ ]:
%%time
mask = pandas.isna(data[["MFLAG", "QFLAG", "SFLAG", "VALUE"]]).sum(axis=1) < 4
data = data[mask]
data